In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# cargando datos de entrenamiento
train_data = np.loadtxt('water_quality_train.csv', delimiter=',', skiprows=1)
X_train, y_train = train_data[:, 0:10], train_data[:, 10]
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", y_train.shape)

X_train shape:  (2293, 10)
Y_train shape:  (2293,)


In [3]:
# cargando datos de test
test_data = np.loadtxt('water_quality_test.csv', delimiter=',', skiprows=1)
X_test = test_data[:, :]
print("X_test shape: ", X_test.shape)

X_test shape:  (983, 10)


In [4]:
# Convertir los datos a tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_train = y_train.view(-1, 1)  # Cambiar la forma de y_train a [n_samples, 1]


# Definir la clase de la red MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.hidden_layer = nn.Linear(input_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        hidden_output = torch.relu(self.hidden_layer(x))
        output = self.output_layer(hidden_output)
        return output

In [5]:

# Parámetros de la red
input_size = 10
hidden_size = 40
output_size = 1

# Crear una instancia de la red
mlp_model = MLP(input_size, hidden_size, output_size)

# Definir función de pérdida y optimizador
criterion = nn.MSELoss()  # Usar error cuadrático medio para tareas de regresión
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# Entrenamiento de la red
num_epochs = 1000

for epoch in range(num_epochs):
    # Paso de adelante y retropropagación
    outputs = mlp_model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Entrenamiento completado.")

Epoch [100/1000], Loss: 116.6100
Epoch [200/1000], Loss: 8.6260
Epoch [300/1000], Loss: 4.7739
Epoch [400/1000], Loss: 2.9144
Epoch [500/1000], Loss: 2.0695
Epoch [600/1000], Loss: 1.6513
Epoch [700/1000], Loss: 1.3722
Epoch [800/1000], Loss: 1.1606
Epoch [900/1000], Loss: 0.9979
Epoch [1000/1000], Loss: 0.8624
Entrenamiento completado.


In [6]:
def predict_probabilities(model, input_data):
    model.eval()
    with torch.no_grad():
        input_tensor = input_data.clone().detach().requires_grad_(False)
        output = model(input_tensor)
        probabilities = torch.sigmoid(output)
    return probabilities

In [7]:
# ...

X_new = X_test[10]  # Ejemplo de nuevos datos de entrada

# Convertir los datos de entrada a tensores PyTorch
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)

predicted_probabilities = predict_probabilities(mlp_model, X_new_tensor)

print("Probabilidades predichas:", predicted_probabilities)

Probabilidades predichas: tensor([0.5789])
